<img src=https://pre00.deviantart.net/4547/th/pre/f/2017/180/1/0/tao_pai_pai_by_wandertof-d9xtmm9.jpg height="500" width="500">

# Baseline model

This is the entrypoint for the competition, it:

* Reads data from tweets' CSV files
* Computes Bag of Words (BoW) from textual representations (tweets text)
* Tests two models to find out which performs better
* Predicts classes for the submission/benchmark tweets
* Generates a suitable CSV for Kaggle InClass

## Data representation

The function `obtain_data_representation` performs the BoW transformation over the training set and applies it to both the train and test set.

If no test set is provided, the input DataFrame is split into both train and test, 75% and 25% of the data respectively. This is done so as to be able to obtain an accuracy score, which will be the evaluation metric on Kaggle.

BoW is computed through `CountVectorizer` class of `sklearn`, restricting it to at most 200 features. The process of finding the best words is done by the `fit` method, whereas transforming the text to numerical vectors (using the learnt features) is done by `transform`. Lastly, `fit_transform` does in a single step the learning and transforming process.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split


def obtain_data_representation(df, test=None):
    # If there is no test data, split the input
    if test is None:
        # Divide data in train and test
        train, test = train_test_split(df, test_size=0.25)
        df.airline_sentiment = pd.Categorical(df.airline_sentiment)
    else:
        # Otherwise, all is train
        train = df
        
    # Create a Bag of Words (BoW), by using train data only
    #cv = CountVectorizer(max_features=200)#, token_pattern=r'[A-Za-z]{3,}|no')
    cv = CountVectorizer(max_features=300,stop_words='english', token_pattern=r'[A-Za-z@#]{3,}|no|yes|wtf|hrs|jfk', min_df = 5)
    
    #max_df=100)
    
    
    #x_train = cv.fit_transform(train['text'])
    #--
    vectonizer = cv.fit(train['text'])
    
    print(cv.vocabulary_)
    x_train = vectonizer.transform(train['text'])
    #--
    
    y_train = train['airline_sentiment'].values
    
    #print(cv.get_feature_names())
    print("vector shape: ", x_train.shape)
    print(type(x_train))
    print(x_train.toarray())
    
    # Obtain BoW for the test data, using the previously fitted one
    x_test = cv.transform(test['text'])
    try:
        y_test = test['airline_sentiment'].values
    except:
        # It might be the submision file, where we don't have target values
        y_test = None
        
    return {
        'train': {
            'x': x_train,
            'y': y_train
        },
        'test': {
            'x': x_test,
            'y': y_test
        }
    }

## Model training

Thought this function might seem strange at first, the only thing to know is that training an `sklearn` model is always done the same way:

```python
# 1. Create the model
model = BernoulliNB()

# 2. Train with some data, where `x` are features and
#    `y` is the target category
model.fit(x, y)

# 3. Predict new categories for test data (with which we
#    have not trained!)
y_pred = model.predict(test_x)
```

We might also obtain the accuracy score by using the function `accuracy_score`

In [8]:
from sklearn.metrics import accuracy_score

def train_model(dataset, dmodel, *model_args, **model_kwargs):
    # Create a Naive Bayes model
    model = dmodel(*model_args, **model_kwargs)
    
    # Train it
    model.fit(dataset['train']['x'], dataset['train']['y'])
    
    # Predict new values for test
    y_pred = model.predict(dataset['test']['x'])
    
    # Print accuracy score unless its the submission dataset
    if dataset['test']['y'] is not None:
        score = accuracy_score(dataset['test']['y'], y_pred)
        print("Model ", dmodel, " score is: {}".format(score))

    # Done
    return model, y_pred

In [9]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier


df = pd.read_csv('tweets_public.csv', index_col='tweet_id')
df = df.loc[df.airline_sentiment_confidence > .99]
dataset = obtain_data_representation(df)

# Train a Bernoulli Naive Bayes
modelNB, _ = train_model(dataset, BernoulliNB)

# Train a K Nearest Neighbors Classifier
modelKN, _ = train_model(dataset, KNeighborsClassifier)

{'@southwestair': 5, 'yes': 298, 'great': 119, '@virginamerica': 8, 'like': 157, 'customer': 67, 'service': 232, '@jetblue': 4, '@united': 6, 'flight': 101, 'delayed': 74, 'hour': 134, 'amp': 20, 'won': 292, 'able': 9, 'tomorrow': 262, 'policy': 203, 'time': 258, 'airport': 17, 'want': 283, 'refund': 213, 'start': 240, '@usairways': 7, 'people': 195, 'working': 294, 'thank': 253, 'flt': 106, 'crew': 66, 'weather': 286, 'delay': 73, 'just': 148, 'good': 117, 'hold': 129, 'http': 137, '@americanair': 2, 'understand': 273, 'answer': 21, 'need': 179, 'home': 130, 'guys': 121, 'thanks': 254, 'help': 126, 'appreciate': 23, 'received': 212, 'bag': 31, 'night': 183, 'missing': 176, 'yesterday': 299, 'stuck': 244, 'disappointed': 83, 'better': 36, 'landed': 150, 'problem': 205, 'minutes': 173, 'cancelled': 48, 'reservation': 215, 'booked': 40, 'hours': 135, 'waited': 281, 'really': 208, 'food': 110, 'don': 87, 'tell': 251, 'actually': 10, 'way': 285, 'ticket': 256, 'check': 54, 'online': 187, '

## Submit file

Once we have found the best model (BernoulliNB for the above simple test), we can train it with all the data (that is, avoid doing a train/test split) and predict sentiments for the real submission data.

This cell below performs exactly this.

In [26]:
import datetime

def create_submit_file(df_submission, ypred):
    date = datetime.datetime.now().strftime("%m_%d_%Y-%H_%M_%S")
    filename = 'submission_' + date + '.csv'
    
    df_submission['airline_sentiment'] = ypred
    df_submission[['airline_sentiment']].to_csv(filename)
    
    print('Submission file created: {}'.format(filename))
    print('Upload it to Kaggle InClass')

    
# Read submission and retrain with whole data
df_submission = pd.read_csv('tweets_submission.csv', index_col='tweet_id')
# We use df_submision as test, otherwise it would split df in train/test
submission_dataset = obtain_data_representation(df, df_submission)
# Predict for df_submission
_, y_pred = train_model(submission_dataset, BernoulliNB)

# Create submission file with obtained y_pred
create_submit_file(df_submission, y_pred)

Submission file created: submission_11_14_2017-19_47_49.csv
Upload it to Kaggle InClass
